# build kubeflow pipeline with python SDK


* Leverage python SDK 
* Create components and pipelines (YAML)

## parameters - user defined

## required installation

In [1]:
!pip install kfp --upgrade

     |████████████████████████████████| 301 kB 9.8 MB/s eta 0:00:01
  Using cached cloudpickle-2.0.0-py3-none-any.whl (25 kB)
  Created wheel for kfp: filename=kfp-1.8.12-py3-none-any.whl size=419048 sha256=837578a7efdf21cc044031ce45450a87757f4520eced6aa4b26f1d2ad43e2fc3
  Stored in directory: /Users/user1/Library/Caches/pip/wheels/e0/a3/8b/897768051e54a03d4ad9d8211e100fc4cc5439d20743d8ab25
Successfully built kfp
  Attempting uninstall: kfp-pipeline-spec
    Found existing installation: kfp-pipeline-spec 0.1.13
    Uninstalling kfp-pipeline-spec-0.1.13:
      Successfully uninstalled kfp-pipeline-spec-0.1.13


  Attempting uninstall: cloudpickle
    Found existing installation: cloudpickle 1.6.0
    Uninstalling cloudpickle-1.6.0:
      Successfully uninstalled cloudpickle-1.6.0
  Attempting uninstall: kfp
    Found existing installation: kfp 1.8.4
    Uninstalling kfp-1.8.4:
      Successfully uninstalled kfp-1.8.4


## import

In [54]:
import kfp
import kfp.components as comp

## 1 Data acquisition and preparation

In [55]:
## load component from yaml
component_download_file = kfp.components.load_component_from_file("./component-sdk-v2.yaml")

def process_data_tarball(file_path: comp.InputPath('Tarball'),
                         output_patient_id_list: comp.OutputPath('CSV'),
                         output_master_df: comp.OutputPath('CSV')):
    """Specific data processing of tarball downloaded.
        - Hard code tarball content names. 
        - Assume there is MANIFEST.txt
        - Output: 
            - output_MANIFEST: MANIFEST.txt as df
            - output_DATA: tables in dict (keys: name as in MANIFEST.txt, values: table)
    
    Args:
        file_path: A string containing path to the tarball.
    """
    from functools import reduce
    import glob
    import json 
    from json import JSONEncoder
    import numpy as np
    import tarfile
    
    import pandas as pd
    
        
    tarfile.open(name=file_path, mode="r|gz").extractall('data_extracted')
    l_tarball_contents = tarfile.open(name=file_path, mode="r|gz").getnames()

    # all dataframes
    d_df_data = {}
    
    for name in l_tarball_contents:
        archive_filename = 'data_extracted/' + name
        if name == 'MANIFEST.txt':
            df_manifest = pd.read_csv(glob.glob(archive_filename)[0], sep = "\t")
        else:
            df = pd.read_csv(glob.glob(archive_filename)[0], sep = "\t")
            df.to_csv(index=False, header=True)
            d_df_data[name] = df    

    filename_patient_table = '57683e22-a8ea-4eca-bfcf-f708cf459546/nationwidechildrens.org_clinical_patient_gbm.txt'
    filename_follow_up_table = 'c9cdbc76-105d-429b-9fce-f000819716f9/nationwidechildrens.org_clinical_follow_up_v1.0_gbm.txt'
            
    # 1 remove un-needed header rows for tables
    df_patient_raw = d_df_data[filename_patient_table]
    df_follow_up_raw = d_df_data[filename_follow_up_table]
    
    ## use the 2nd row as the column name (in raw data, first 3 rows are column labels)
    ### patient table
    df_patient = df_patient_raw.drop([1])
    df_patient = df_patient.tail(len(df_patient) -1 )
    df_patient = df_patient.reset_index().drop(['index'], axis = 1)
    ### follow_up table
    df_follow_up = df_follow_up_raw.drop([1])
    df_follow_up = df_follow_up.tail(len(df_follow_up) -1 )
    df_follow_up = df_follow_up.reset_index().drop(['index'], axis = 1)
    
    
    # 2 data processing
    ## 2a: data processing: clnical data
    
    ### patient table
    missing_value_flags = ['[Not Available]',
                       '[Discrepancy]',
                       '[Unknown]',
                       '[Not Applicable]',
                       '[Not Evaluated]'
                      ]
    #### replace flags with np.nan
    for flag in missing_value_flags:
        df_patient = df_patient.replace(flag, np.nan)
    
    #### drop un-needed columns - identifiers
    df_patient = df_patient.drop(['bcr_patient_uuid', 'form_completion_date', 'patient_id'], axis=1)
    
    #### parse numerical columns w/ dates
    l_numerical_cols = [
        'last_contact_days_to',
        'death_days_to',
        'age_at_initial_pathologic_diagnosis'
    ]
    for col in l_numerical_cols:
        df_patient[col] = [float(x) for x in df_patient[col]]

    df_patient['survival_time_yrs'] = np.abs(df_patient['death_days_to']) / 365
    df_patient['days_since_last_contact'] = np.abs(df_patient['last_contact_days_to']) / 365
    
    #### drop columns parsed / not needed anymore
    l_columns_not_needed = [
        'last_contact_days_to',
        'birth_days_to',
        'death_days_to',

        # uninformative cols (all missing/not evaluated etc):
        'anatomic_neoplasm_subdivision',
        'disease_code',
        'project_code',
        'days_to_initial_pathologic_diagnosis',
        'icd_10',
        'icd_o_3_histology',
        'icd_o_3_site',
        'informed_consent_verified',
        'initial_pathologic_dx_year'

    ]
    df_patient = df_patient.drop(l_columns_not_needed, axis = 1)
    
    
    
    ### follow_up table
    #### replace missing value flags with np.nan
    missing_value_flags = ['[Not Available]',
                           '[Discrepancy]',
                           '[Unknown]',
                           '[Not Applicable]',
                           '[Not Evaluated]'
                          ]
    for flag in missing_value_flags:
        df_follow_up = df_follow_up.replace(flag, np.nan)

    #### drop un-needed columns - identifiers
    df_follow_up = df_follow_up.drop(['bcr_patient_uuid', 'bcr_followup_uuid', 'form_completion_date',
                                     'followup_reason', 'followup_lost_to'], axis=1)

    #### parse numerical columns w/ dates
    l_numerical_cols = [
        'last_contact_days_to',
        'death_days_to'
    ]
    for col in l_numerical_cols:
        df_follow_up[col] = [float(x) for x in df_follow_up[col]]

    #### isolate the LAST followup (sorted by barcode) --> get one row per patient
    df_follow_up = df_follow_up.groupby(['bcr_patient_barcode']).tail(1)

    #### drop columns parsed / not needed anymore
    l_columns_not_needed = [
        'last_contact_days_to',
        'death_days_to',
        'bcr_followup_barcode'
    ]
    df_follow_up = df_follow_up.drop(l_columns_not_needed, axis = 1)
    
    
    # 3 Merge tables by patient identifier (create master table)

    ## merge patient and followup table
    ### find common cols: 
    l_cols_table_1 = set(df_patient.columns)
    l_cols_table_2 = set(df_follow_up.columns)
    intersect_cols = l_cols_table_1.intersection(l_cols_table_2)
    l_cols_to_remove_table_1 = [x for x in intersect_cols if x != 'bcr_patient_barcode']

    ### drop common cols from left table (`patient`) before join
    df_patient = df_patient.drop(l_cols_to_remove_table_1, axis = 1)
    ### join
    df_master = df_patient.merge(df_follow_up, 
                                 left_on = 'bcr_patient_barcode', 
                                 right_on = 'bcr_patient_barcode', 
                                 how = 'inner')
    ### drop patient ID identifier from master table (used as identifier, not treated as a feature)
    df_patient_id_list = df_master[['bcr_patient_barcode']]
    df_master = df_master.drop(['bcr_patient_barcode'], axis = 1)    
        
    # pl output
    df_patient_id_list.to_csv(output_patient_id_list, index=False, header=True)
    df_master.to_csv(output_master_df, index=False, header=True)
    
        
    
create_step_process_data_tarball = kfp.components.create_component_from_func(
    func=process_data_tarball,
    output_component_file='component_process_data_tarball.yaml', # save the component spec for future use.
    base_image='python:3.7',
    packages_to_install=['pandas==1.1.4'])

## 2 Data processing

Further process master table

- select column to be used as class label; 
- for other columns, get dummies

In [463]:
def get_dummies(file_path: comp.InputPath('CSV'),
                class_label_colname: str,
                s_colnames_to_exclude: str,
                output_csv_features: comp.OutputPath('CSV'),
                output_csv_target: comp.OutputPath('CSV'),
                output_csv_target_class_labels: comp.OutputPath('CSV'),
                output_csv_feature_list: comp.OutputPath('CSV')):
    """Distribute categorical features into separate features.
        Input: CSV with categorical (and numeric) features. Assume last 
            feature is target label. 
        Output: CSV with categorical features separated into dummies.
    
    Args:
        file_path: A string containing path to input data.
        output_csv: A string containing path to processed data.
    """
    import glob
    import numpy as np
    import pandas as pd
    from sklearn import preprocessing
    
    df = pd.read_csv(filepath_or_buffer=file_path)
    l_col_names = list(df.columns)
    
    # isolate column for target class label; if null, use last column
    target_class_label = class_label_colname if str(class_label_colname) != '' else l_col_names[-1]
    ## remove all rows where class label col is NaN
    df = df.dropna(subset=[target_class_label])
    
    ## extract target class column
    df_target_raw = df[target_class_label] # from input parameter
    lb = preprocessing.LabelBinarizer()
    lb.fit(df_target_raw.astype(str)) # fit to data for target class to find all classes
    target_class_label_names = lb.classes_ # store array of all the classes
#    df_target = pd.DataFrame(np.array(lb.fit_transform(df_target_raw)))

    df_target_class_labels = pd.DataFrame({'class': target_class_label_names})
    d_target_class_label_idx = dict(zip(list(df_target_class_labels['class']), list(df_target_class_labels.index)))
    
    l_target_column = [d_target_class_label_idx[x] for x in df[target_class_label]]
    df_target = pd.DataFrame([])
    df_target[target_class_label] = l_target_column# 1-column of multiple classes
    

    
    # create dummies for every col except class label col
    df_features = df[[x for x in l_col_names if x != class_label_colname]] # features are all colnames except target class
    
    # exclude any other colnames (other than target class column), if specified
    if str(s_colnames_to_exclude) != '':
        l_colnames_to_exclude = s_colnames_to_exclude.split(',')
        l_colnames_in_this_df_to_exclude = [x for x in l_colnames_to_exclude if x in df.columns]
        df.drop(l_colnames_to_exclude, axis = 1)
    
    # get dummies
    df_features_dummies = pd.get_dummies(df_features)
    
    # feature list
    l_features = df_features_dummies.columns
    df_feature_list = pd.DataFrame({'feature': l_features})
        
    # write outputs
    df_features_dummies.to_csv(output_csv_features, index = False, header = True)
    df_target.to_csv(output_csv_target, index = False, header = True)
    df_target_class_labels.to_csv(output_csv_target_class_labels, index = True, header = False)
    df_feature_list.to_csv(output_csv_feature_list, index = True, header = False)

create_step_dp_get_dummies = kfp.components.create_component_from_func(
    func=get_dummies,
    output_component_file='component_dp_get_dummies.yaml', # save the component spec for future use.
    base_image='python:3.7',
    packages_to_install=['pandas==1.1.4',
                        'scikit-learn==1.0.2'])

### Component: Imputation

In [296]:
def impute_unknown(file_path: comp.InputPath('CSV'),
                   output_csv: comp.OutputPath('CSV')):
    """Impute unknown values (nan).
        Input: CSV.
        Output: CSV.
    
    Args:
        file_path: A string containing path to input data.
        output_csv: A string containing path to processed data.
    """
    import numpy as np
    import pandas as pd
    from sklearn.impute import SimpleImputer
    
    # Read in CSV
    df = pd.read_csv(filepath_or_buffer=file_path)
    
    # Impute: most common
    imp_most_frequent = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
    imp_most_frequent.fit(df)
    nparr_imputed = imp_most_frequent.transform(df)
    df_imputed = pd.DataFrame(nparr_imputed)
    df_imputed.columns = df.columns
    
    # Output to CSV
    df_imputed.to_csv(output_csv, index = False, header = True)

create_step_dp_impute_unknown = kfp.components.create_component_from_func(
    func=impute_unknown,
    output_component_file='component_dp_impute_unknown.yaml', # save the component spec for future use.
    base_image='python:3.7',
    packages_to_install=['pandas==1.1.4',
                         'scikit-learn==1.0.2'])

### Component: Scaler

In [268]:
def scale_df(file_path: comp.InputPath('CSV'),
             output_csv: comp.OutputPath('CSV')):
    """Impute unknown values (nan).
        Input: CSV.
        Output: CSV.
    
    Args:
        file_path: A string containing path to input data.
        output_csv: A string containing path to processed data.
    """
    import pandas as pd
    from sklearn.preprocessing import StandardScaler
    
    # Read in CSV
    df = pd.read_csv(filepath_or_buffer=file_path)
    
    # scaler
    scaler = StandardScaler()
    scaler.fit(df)
    nparr_scaled_data = scaler.transform(df)
    
    df_scaled = pd.DataFrame(nparr_scaled_data)
    df_scaled.columns = df.columns
    
    # Output to CSV
    df_scaled.to_csv(output_csv, index = False, header = True)

create_step_dp_scale_df = kfp.components.create_component_from_func(
    func=scale_df,
    output_component_file='component_dp_scale_df.yaml', # save the component spec for future use.
    base_image='python:3.7',
    packages_to_install=['pandas==1.1.4',
                        'scikit-learn==1.0.2'])

### Component: create model inputs: X,y

In [240]:
def create_model_inputs(file_path_features: comp.InputPath('CSV'),
                        file_path_class_labels: comp.InputPath('CSV'),
                        test_set_size: str,
                        output_X_train: comp.OutputPath('CSV'),
                        output_X_test: comp.OutputPath('CSV'),
                        output_y_train: comp.OutputPath('CSV'),
                        output_y_test: comp.OutputPath('CSV')):
    """Create train and test sets
    
    Args:
        file_path_features: A string containing path / pipeline component output
        file_path_class_labels: A string containing path to data / pipeline component output
        test_set_size: [optional, default=0.25] proportion to use for the test set
    """
    import numpy as np
    
    import pandas as pd
    from sklearn import datasets
    from sklearn.model_selection import train_test_split

    X = pd.read_csv(filepath_or_buffer=file_path_features)
    y = pd.read_csv(filepath_or_buffer=file_path_class_labels)
    
    try:
        test_set_size_for_model_training = float(test_set_size)
    except:
        test_set_size_for_model_training = 0.25
    if test_set_size_for_model_training <= 0 or test_set_size_for_model_training >= 1:
        test_set_size_for_model_training = 0.25
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_set_size_for_model_training)
    
    X_train.to_csv(output_X_train, header=True, index=False)
    X_test.to_csv(output_X_test, header=True, index=False)
    y_train.to_csv(output_y_train, header=True, index=False)
    y_test.to_csv(output_y_test, header=True, index=False)

    
create_step_dp_create_train_test = kfp.components.create_component_from_func(
    func=create_model_inputs,
    output_component_file='component_dp_create_train_test.yaml', # save the component spec for future use.
    base_image='python:3.7',
    packages_to_install=['pandas==1.1.4',
                         'scikit-learn==1.0.2'])

## 3 model train

In [357]:
def train_model(file_path_x_train: comp.InputPath('CSV'), 
                file_path_y_train: comp.InputPath('CSV'),
                output_model: comp.OutputPath('Model')):
    import joblib
    import mlflow.sklearn
    import pandas as pd
    from sklearn.linear_model import LogisticRegression
    from sklearn import preprocessing

    
    X_train = pd.read_csv(file_path_x_train)
    y_train_raw = pd.read_csv(file_path_y_train)
    
    
#     lb = preprocessing.LabelBinarizer()
#     lb.fit_transform(y_train_raw.astype(str))
#     y_class_labels = lb.classes_ #
#     print('y_class_labels: ', y_class_labels) #
#     y_train = lb.fit_transform(y_train_raw.astype(str))    
    
    y_train=y_train_raw
    
    model = LogisticRegression(verbose=1,
                               penalty='l2',
                               tol=1e-4,
                               C=1.0,
                               class_weight='balanced',
                               solver='lbfgs',
                               multi_class='ovr')
    model.fit(X_train, y_train)

    
    mlflow.sklearn.save_model(model, output_model,
                          serialization_format=mlflow.sklearn.SERIALIZATION_FORMAT_PICKLE)

    # log model
    mlflow.sklearn.log_model(model, "sklearn_models")

create_step_train_model = kfp.components.create_component_from_func(
    func=train_model,
    output_component_file='component_train_model.yaml', 
    base_image='python:3.7',
    packages_to_install=['joblib==1.1.0',
                         'mlflow==1.24.0',
                         'pandas==1.1.4',
                         'scikit-learn==1.0.2'])

## 4 Test

In [483]:
def test_model(file_path_x_test: comp.InputPath('CSV'),
               file_path_y_test: comp.InputPath('CSV'),
               file_path_model: comp.InputPath('Model'),
               output_json: comp.OutputPath('JSON'),
               output_csv:  comp.OutputPath('CSV')):
    import joblib
    import json
    import mlflow.sklearn
    import numpy as np
    import pandas as pd
    from sklearn.linear_model import LogisticRegression
    from sklearn.metrics import accuracy_score
    from sklearn.metrics import confusion_matrix
    from sklearn.metrics import roc_auc_score
    from sklearn.preprocessing import LabelBinarizer

    class NumpyArrayEncoder(json.JSONEncoder):
        def default(self, obj):
            if isinstance(obj, np.ndarray):
                return obj.tolist()
            return json.JSONEncoder.default(self, obj)
        
    trained_model = mlflow.sklearn.load_model(file_path_model)
    
    X_test = pd.read_csv(file_path_x_test)
    y_test = np.array(pd.read_csv(file_path_y_test))
    
    # Get predictions 
    y_pred = trained_model.predict(X_test)
    
    
    #############################################################
    
    
    # Get accuracy
    accuracy = accuracy_score(y_test, y_pred)
    
    # Confusion matrix - use labels
    cm = confusion_matrix(y_test, y_pred)
    
    # AUC score
    sum_y_test_axis_0 = np.sum(y_test, axis=0)
    col_idx_y_test_nnz = np.where(sum_y_test_axis_0 > 0)[0]
    
    # compute AUC 

    ## check if multi-class
    num_classes = np.max([len(np.unique(y_pred)), len(np.unique(y_test))])
    if num_classes > 2: # one-hot encode
        lb = LabelBinarizer()
        lb.fit(np.concatenate((np.array(y_test).ravel(), np.array(y_pred).ravel())))
        y_test_onehot = lb.fit_transform(y_test)
        y_pred_onehot = lb.fit_transform(y_pred)
    else:
        y_test_onehot = y_test
        y_pred_onehot = y_pred
        
    auc_test = roc_auc_score(y_test_onehot,
                             y_pred_onehot,
                             multi_class=trained_model.multi_class) # specify multi-class method

    # output: JSON
    d_output = {}
    ## add in model results
    d_output['model_results'] = {}
    d_output['model_results']['model'] = trained_model.get_params()
    d_output['model_results']['y_test'] = np.array(y_test).ravel()
    d_output['model_results']['y_pred'] = y_pred
    d_output['model_results']['accuracy'] = accuracy
    d_output['model_results']['auc_test'] = auc_test
    d_output['model_results']['cm'] = cm

    json_string_output = json.dumps(d_output, cls=NumpyArrayEncoder)
    
    ## write json output: results
    with open(output_json, 'w') as outfile:
        outfile.write(json_string_output)
        
    ## write csv output: results
    df_output_csv = pd.DataFrame({'metric': d_output['model_results'].keys(),
                                  'value':  d_output['model_results'].values()})
    df_output_csv.to_csv(output_csv, index = False, header = True)
    
create_step_test_model = kfp.components.create_component_from_func(
    func=test_model,
    output_component_file='component_test_model.yaml', 
    base_image='python:3.7',
    packages_to_install=['joblib==1.1.0',
                         'mlflow==1.24.0',
                         'pandas==1.1.4',
                         'scikit-learn==1.0.2'])

## 5 Performance Analysis

### pl04 Final Pipeline: data process, train, test


In [484]:
def pl_GBM_04_webdl_dp_train(user_input_class_label_column_name,
                            s_colnames_to_exclude,
                            test_set_size):
    """Pipeline: Download data, data processing
        1. download data and make master table
        2. data processing
        3. train model
    
    Args:
        user_input_class_label_column_name: name of column to use as class label (i.e., vital_status)
        s_colnames_to_exclude: other columns to exclude, other than the ones automatically 
                               filtered out in make_dfs(); separate colnames by ','
    """
    url = 'https://wiki.cancerimagingarchive.net/download/attachments/1966258/gdc_download_clinical_gbm.tar.gz'
    web_downloader_task = component_download_file(url)
    make_dfs = create_step_process_data_tarball(file=web_downloader_task.outputs['data'])
    dp_get_dummies_task = create_step_dp_get_dummies(file=make_dfs.outputs['output_master_df'],
                                                     class_label_colname=user_input_class_label_column_name, 
                                                     s_colnames_to_exclude=s_colnames_to_exclude)
    dp_impute_task = create_step_dp_impute_unknown(file=dp_get_dummies_task.outputs['output_csv_features'])
    dp_scale_task = create_step_dp_scale_df(file=dp_impute_task.outputs['output_csv'])
    dp_create_train_test = create_step_dp_create_train_test(file_path_features=dp_scale_task.outputs['output_csv'], 
                                                            file_path_class_labels=dp_get_dummies_task.outputs['output_csv_target'],
                                                            test_set_size=test_set_size)
    train_model_task = create_step_train_model(file_path_x_train=dp_create_train_test.outputs['output_X_train'],
                                               file_path_y_train=dp_create_train_test.outputs['output_y_train'])
    test_model_task = create_step_test_model(file_path_x_test=dp_create_train_test.outputs['output_X_test'],
                                             file_path_y_test=dp_create_train_test.outputs['output_y_test'],
                                             file_path_model=train_model_task.outputs['output_model'])
    
kfp.compiler.Compiler().compile(
    pipeline_func=pl_GBM_04_webdl_dp_train,
    package_path='pl_GBM_04_webdl_dp_train_test.yaml')